# EV Battery Digital Twin Analysis
## TATA Technologies Hackathon Project

This notebook demonstrates the battery digital twin functionality and shows how to predict battery aging, remaining useful life, and optimize charging strategies.

In [ ]:
import sys
import os
sys.path.append('../src')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

from digital_twin.battery_twin import BatteryDigitalTwin, BatteryState
from utils.visualization import EVVisualization, EVMetrics, EVUtils

# Configure plotting
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
%matplotlib inline

## 1. Initialize Battery Digital Twin

Let's create a digital twin for a typical TATA EV battery pack.

In [ ]:
# Initialize battery digital twin for TATA EV
tata_battery = BatteryDigitalTwin(battery_capacity=75.0, chemistry="Li-ion")

print(f"Battery Digital Twin Initialized:")
print(f"- Capacity: {tata_battery.battery_capacity} kWh")
print(f"- Chemistry: {tata_battery.chemistry}")
print(f"- Calendar aging factor: {tata_battery.calendar_aging_factor}")
print(f"- Cycle aging factor: {tata_battery.cycle_aging_factor}")

## 2. Current Battery State Analysis

Let's analyze the current state of a battery that has been in use for 2 years.

In [ ]:
# Create current battery state (2 years old)
current_state = BatteryState(
    soh=0.92,  # Will be recalculated
    soc=0.65,  # 65% charge
    temperature=25.0,  # 25°C
    voltage=3.7,
    current=0.0,
    cycle_count=800,  # ~1 cycle per day for 2+ years
    age_days=730  # 2 years
)

# Calculate actual SOH
actual_soh = tata_battery.calculate_soh(current_state)
current_state.soh = actual_soh

print(f"Battery Analysis (2 years old):")
print(f"- Calculated SOH: {actual_soh:.3f} ({actual_soh*100:.1f}%)")
print(f"- Current SOC: {current_state.soc*100:.1f}%")
print(f"- Temperature: {current_state.temperature}°C")
print(f"- Total cycles: {current_state.cycle_count}")
print(f"- Age: {current_state.age_days} days ({current_state.age_days/365:.1f} years)")

## 3. Comprehensive Health Report

Generate a detailed health report including recommendations.

In [ ]:
# Generate comprehensive health report
health_report = tata_battery.get_health_report(current_state)

print("=" * 60)
print("             BATTERY HEALTH REPORT")
print("=" * 60)
print(f"Health Grade: {health_report['health_grade']}")
print(f"State of Health: {health_report['state_of_health']*100:.1f}%")
print(f"Remaining Capacity: {health_report['remaining_capacity_kwh']:.1f} kWh")
print(f"Remaining Useful Life: {health_report['remaining_useful_life_days']} days ({health_report['remaining_useful_life_days']/365:.1f} years)")
print(f"Temperature Status: {health_report['temperature_status']}")

print("\n--- CHARGING OPTIMIZATION ---")
charging_opt = health_report['charging_optimization']
print(f"Recommended Charge Rate: {charging_opt['charge_rate_c']:.1f}C")
print(f"Target SOC: {charging_opt['target_soc']*100:.0f}%")
print(f"Estimated Charging Time: {charging_opt['estimated_time_hours']:.1f} hours")

print("\n--- SECOND-LIFE POTENTIAL ---")
second_life = health_report['second_life_potential']
print(f"Recommended Application: {second_life['recommended_application']}")
print(f"Suitability Rating: {second_life['suitability_rating']}")
print(f"Estimated Second-Life Years: {second_life['estimated_second_life_years']}")

print("\n--- RECOMMENDATIONS ---")
for i, rec in enumerate(health_report['recommendations'], 1):
    print(f"{i}. {rec}")

## 4. Battery Life Simulation

Simulate battery degradation over the next 10 years under different conditions.

In [ ]:
# Simulate battery life under different scenarios
scenarios = {
    "Conservative Use": {"daily_cycles": 0.8, "avg_temperature": 20},
    "Normal Use": {"daily_cycles": 1.2, "avg_temperature": 25},
    "Heavy Use": {"daily_cycles": 2.0, "avg_temperature": 30}
}

simulation_results = {}

for scenario_name, params in scenarios.items():
    sim_data = tata_battery.simulate_battery_life(
        years=10,
        daily_cycles=params["daily_cycles"],
        avg_temperature=params["avg_temperature"]
    )
    simulation_results[scenario_name] = sim_data
    
    print(f"\n{scenario_name} Scenario:")
    print(f"- Daily cycles: {params['daily_cycles']}")
    print(f"- Average temperature: {params['avg_temperature']}°C")
    print(f"- SOH after 5 years: {sim_data[sim_data['month'] == 60]['soh'].iloc[0]:.3f}")
    print(f"- SOH after 10 years: {sim_data[sim_data['month'] == 120]['soh'].iloc[0]:.3f}")
    print(f"- Capacity after 10 years: {sim_data[sim_data['month'] == 120]['capacity_kwh'].iloc[0]:.1f} kWh")

## 5. Visualization: Battery Degradation Comparison

In [ ]:
# Create comparison plot
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))

# Plot 1: SOH over time
for scenario, data in simulation_results.items():
    years = data['month'] / 12
    ax1.plot(years, data['soh'] * 100, label=scenario, linewidth=2)

ax1.set_xlabel('Years')
ax1.set_ylabel('State of Health (%)')
ax1.set_title('Battery Health Degradation Over Time')
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.axhline(y=80, color='red', linestyle='--', alpha=0.7, label='End of Life Threshold')

# Plot 2: Remaining capacity
for scenario, data in simulation_results.items():
    years = data['month'] / 12
    ax2.plot(years, data['capacity_kwh'], label=scenario, linewidth=2)

ax2.set_xlabel('Years')
ax2.set_ylabel('Remaining Capacity (kWh)')
ax2.set_title('Battery Capacity Over Time')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Plot 3: Cycle count impact
normal_data = simulation_results["Normal Use"]
ax3.scatter(normal_data['cycle_count'], normal_data['soh'] * 100, alpha=0.6, s=30)
ax3.set_xlabel('Total Cycle Count')
ax3.set_ylabel('State of Health (%)')
ax3.set_title('Cycle Count vs Battery Health')
ax3.grid(True, alpha=0.3)

# Plot 4: RUL over time
for scenario, data in simulation_results.items():
    years = data['month'] / 12
    rul_years = data['rul_days'] / 365
    ax4.plot(years, rul_years, label=scenario, linewidth=2)

ax4.set_xlabel('Years')
ax4.set_ylabel('Remaining Useful Life (Years)')
ax4.set_title('Remaining Useful Life Prediction')
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.suptitle('TATA EV Battery Digital Twin Analysis', fontsize=16, y=1.02)
plt.show()

## 6. Charging Optimization Analysis

Analyze optimal charging strategies under different conditions.

In [ ]:
# Analyze charging optimization for different scenarios
charging_scenarios = [
    {"soc": 0.2, "temp": -5, "target": 0.8, "condition": "Cold Weather"},
    {"soc": 0.3, "temp": 25, "target": 0.8, "condition": "Normal Weather"},
    {"soc": 0.1, "temp": 40, "target": 0.9, "condition": "Hot Weather"},
    {"soc": 0.5, "temp": 20, "target": 1.0, "condition": "Long Trip Prep"}
]

charging_analysis = []

for scenario in charging_scenarios:
    strategy = tata_battery.optimize_charging_strategy(
        current_soc=scenario["soc"],
        target_soc=scenario["target"],
        temperature=scenario["temp"]
    )
    
    analysis = {
        "Condition": scenario["condition"],
        "Start SOC": f"{scenario['soc']*100:.0f}%",
        "Target SOC": f"{scenario['target']*100:.0f}%",
        "Recommended Target": f"{strategy['target_soc']*100:.0f}%",
        "Temperature": f"{scenario['temp']}°C",
        "Charge Rate": f"{strategy['charge_rate_c']:.1f}C",
        "Max Voltage": f"{strategy['max_voltage']:.2f}V",
        "Est. Time": f"{strategy['estimated_time_hours']:.1f}h",
        "Temp Compensation": "Yes" if strategy['temperature_compensation'] else "No"
    }
    
    charging_analysis.append(analysis)

# Display as DataFrame
charging_df = pd.DataFrame(charging_analysis)
print("CHARGING OPTIMIZATION ANALYSIS")
print("=" * 80)
print(charging_df.to_string(index=False))

## 7. Second-Life Battery Applications

Analyze different batteries at various SOH levels for second-life applications.

In [ ]:
# Analyze second-life potential for batteries at different SOH levels
soh_levels = np.arange(0.5, 1.01, 0.05)
second_life_analysis = []

for soh in soh_levels:
    # Create battery state with given SOH
    test_state = BatteryState(
        soh=soh,
        soc=0.7,
        temperature=25,
        voltage=3.7,
        current=0.0,
        cycle_count=int((1-soh) * 2000),  # Approximate cycle count from SOH
        age_days=int((1-soh) * 3650)  # Approximate age from SOH
    )
    
    second_life = tata_battery.assess_second_life_potential(test_state)
    
    second_life_analysis.append({
        'SOH': soh,
        'SOH_Percent': soh * 100,
        'Remaining_Capacity': second_life['remaining_capacity_kwh'],
        'Application': second_life['recommended_application'],
        'Suitability': second_life['suitability_rating'],
        'Second_Life_Years': second_life['estimated_second_life_years']
    })

second_life_df = pd.DataFrame(second_life_analysis)

# Create visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Plot 1: SOH vs Applications
app_colors = {
    'EV Use - Continue': 'green',
    'Grid Storage': 'blue',
    'Home Energy Storage': 'orange',
    'UPS/Backup Power': 'red',
    'Material Recovery': 'gray'
}

for app in app_colors.keys():
    app_data = second_life_df[second_life_df['Application'] == app]
    if not app_data.empty:
        ax1.scatter(app_data['SOH_Percent'], app_data['Remaining_Capacity'], 
                   label=app, color=app_colors[app], s=60, alpha=0.7)

ax1.set_xlabel('State of Health (%)')
ax1.set_ylabel('Remaining Capacity (kWh)')
ax1.set_title('Second-Life Applications by Battery Health')
ax1.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
ax1.grid(True, alpha=0.3)

# Plot 2: Second-life years potential
ax2.plot(second_life_df['SOH_Percent'], second_life_df['Second_Life_Years'], 
         'o-', linewidth=2, markersize=6, color='purple')
ax2.set_xlabel('State of Health (%)')
ax2.set_ylabel('Estimated Second-Life Years')
ax2.set_title('Second-Life Potential Duration')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Summary table
print("\nSECOND-LIFE APPLICATION MAPPING")
print("=" * 60)
summary = second_life_df.groupby('Application').agg({
    'SOH_Percent': ['min', 'max'],
    'Remaining_Capacity': ['min', 'max'],
    'Second_Life_Years': 'mean'
}).round(1)

print(summary)

## 8. Key Insights and Conclusions

### Digital Twin Capabilities:
1. **Accurate SOH Prediction**: The model incorporates calendar aging, cycle aging, temperature effects, and depth of discharge
2. **Remaining Useful Life**: Provides realistic RUL estimates for maintenance planning
3. **Charging Optimization**: Adapts charging strategy based on temperature and battery condition
4. **Second-Life Assessment**: Identifies optimal reuse applications for degraded batteries

### Business Impact for TATA:
1. **Predictive Maintenance**: Reduce unexpected failures and optimize service schedules
2. **Customer Confidence**: Provide accurate battery health information to customers
3. **Sustainability**: Enable circular economy through effective battery reuse
4. **Cost Optimization**: Extend battery life through optimal charging strategies

### Technical Achievements:
- Physics-based degradation modeling
- Multi-factor optimization algorithms
- Real-time battery state assessment
- Comprehensive second-life evaluation framework